
---

### What Are MCS Locks?

MCS locks are a **spinlock variant** introduced by John M. Mellor-Crummey and Michael L. Scott in 1989. They are designed to address the problem of **cache-line bouncing** and poor scalability in traditional spinlocks, where multiple threads busy-wait (spin) on a single shared memory location, causing excessive cache coherence traffic. MCS locks achieve better performance in high-contention scenarios by organizing waiting threads into a queue, where each thread spins on a **local variable** in its own memory, reducing cache contention.

#### Key Features of MCS Locks:
1. **Queue-Based Locking**:
   - Threads attempting to acquire the lock form a queue, with each thread linked to the next.
   - Only the thread at the head of a queue holds the lock; others wait their turn.
2. **Local Spinning**:
   - Each waiting thread spins on a **local flag** (usually in its own cache line), avoiding contention on a single shared lock variable.
3. **Fairness**:
   - MCS locks are **FIFO (First-In-First-Out)**, ensuring threads acquire the lock in the order they requested it, preventing starvation.
4. **Scalability**:
   - By reducing cache-line bouncing, MCS locks perform well in systems with many cores or threads compared to simple test-and-set spinlocks.

---

### How MCS Locks Work

An MCS lock consists of two main components:
1. **Lock Structure**: A pointer to the tail of the queue of waiting threads.
2. **Per-Thread Node**: Each thread has a node (a structure) containing:
   - A ```:disable-run
`next`
``` pointer to the next thread’s node in the queue.
   - A ```:disable-run
`locked`
``` flag indicating whether the thread has acquired the lock.

#### Lock Acquisition:
1. A thread creates its own node with ```:disable-run
`locked = true`
``` (indicating it’s waiting) and ```:disable-run
`next = NULL`
```.
2. The thread atomically swaps its node with the lock’s ```:disable-run
`tail`
``` pointer using a ```:disable-run
`compare-and-swap`
``` (CAS) or similar operation.
3. If the previous ```:disable-run
`tail`
``` was ```:disable-run
`NULL`
```, the thread acquires the lock immediately (```:disable-run
`locked`
``` is set to ```:disable-run
`false`
```).
4. Otherwise, the thread links itself to the previous tail’s ```:disable-run
`next`
``` pointer and spins on its own ```:disable-run
`locked`
``` flag, waiting for the previous thread to release the lock.

#### Lock Release:
1. The thread checks if its node’s ```:disable-run
`next`
``` pointer is ```:disable-run
`NULL`
``` (no waiting threads).
2. If ```:disable-run
`next`
``` is ```:disable-run
`NULL`
```, it atomically sets the lock’s ```:disable-run
`tail`
``` to ```:disable-run
`NULL`
``` using CAS, indicating the lock is free.
3. If ```:disable-run
`next`
``` is not ```:disable-run
`NULL`
```, the thread sets the ```:disable-run
`locked`
``` flag of the next thread’s node to ```:disable-run
`false`
```, signaling that the next thread can acquire the lock.

#### Pseudo-Code Example:
```c
struct mcs_node {
    struct mcs_node *next;
    bool locked;
};

struct mcs_lock {
    struct mcs_node *tail;
};

void mcs_lock(struct mcs_lock *lock, struct mcs_node *my_node) {
    my_node->next = NULL;
    my_node->locked = true;
    // Atomically swap my_node with lock->tail
    struct mcs_node *prev = atomic_exchange(&lock->tail, my_node);
    if (prev != NULL) {
        // Queue is non-empty, link to previous node
        prev->next = my_node;
        // Spin on local flag
        while (my_node->locked) { /* busy wait */ }
    }
    // Lock acquired
}

void mcs_unlock(struct mcs_lock *lock, struct mcs_node *my_node) {
    if (my_node->next == NULL) {
        // Try to clear the tail
        if (atomic_compare_exchange(&lock->tail, &my_node, NULL)) {
            return; // No waiters, lock is free
        }
        // Wait for next node to be linked
        while (my_node->next == NULL) { /* busy wait */ }
    }
    // Signal the next thread
    my_node->next->locked = false;
}